In [ ]:
pip install -U transformers praw torch numpy pandas

In [5]:
import pandas as pd
import praw
from praw.models import MoreComments
from getpass import getpass

In [22]:
client_id = getpass('Enter client_id: ')

In [23]:
client_secret = getpass('Enter client_secret: ')

In [24]:
password = getpass('Enter password: ')

In [26]:
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=password,
    user_agent="testscript by u/curtpond",
    username="curtpond",
)

In [27]:
print(reddit.read_only)

False


In [28]:
reddit.read_only = True

In [32]:
# Get the top 10 hot posts from the MachineLearning subreddit
subreddit = reddit.subreddit('GRE')
 
# display the subreddit name
print(subreddit.display_name)
 
# display the subreddit title
print(subreddit.title)      
 
# display the subreddit description
print(subreddit.description)

GRE
r/GRE
#/r/GRE  

This subreddit is for discussion of the GRE (Graduate Record Examination). If you're studying for the GRE, or can help people who are studying for the GRE, you're in the right place!

&nbsp; 

-----

#Rules

- You must read and follow the rules! 
https://www.reddit.com/r/gre/about/rules

-----


In [31]:
hot_posts = reddit.subreddit('MachineLearning').hot(limit=10)
for post in hot_posts:
    print(post.title)

[D] Simple Questions Thread
[D] Machine Learning - WAYR (What Are You Reading) - Week 140
[P] Implementation of MagicMix from ByteDance researchers, - New way to interpolate concepts with much more natural, geometric coherency (implemented with Stable Diffusion!)
[R] Code as Policies: Language Model Programs for Embodied Control - Robotics at Google 2022 - Robots that write their own code
[D] Graph neural networks
[D] What are the benefits of being a reviewer?
[R] ICML 2022 Paper Summaries (HUMAN)
[N] Adversarial Policies Beat Professional-Level Go AIs
[N] On the detection of synthetic images generated by diffusion models
[R] Instance segmentation of paper fiber networks


In [35]:
import pandas as pd
posts = []
ml_subreddit = reddit.subreddit('MachineLearning')
for post in ml_subreddit.hot(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts.head(10)

,title,score,id,subreddit,url,num_comments,body,created
0,[D] Simple Questions Thread,10,ybjvk5,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,78,Please post your questions here instead of cre...,1.666537e+09
1,[D] Machine Learning - WAYR (What Are You Read...,199,vg5kjd,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,57,This is a place to share machine learning rese...,1.655675e+09
2,[P] Implementation of MagicMix from ByteDance ...,49,ykiuq0,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,5,Hi. Today I've came across this interesting pa...,1.667426e+09
3,[R] Code as Policies: Language Model Programs ...,20,ykm5rc,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,Paper: [https://arxiv.org/abs/2209.07753](http...,1.667433e+09
4,[D] Graph neural networks,41,ykajrg,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,18,I have never used GNNs before. I was wondering...,1.667408e+09
5,[D] What are the benefits of being a reviewer?,29,ykbp1s,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,20,So I just got invited to serve as a reviewer f...,1.667411e+09
6,[R] ICML 2022 Paper Summaries (HUMAN),3,ykqh58,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,0,"It's been a while since ICML 2022, but here ar...",1.667444e+09
7,[N] Adversarial Policies Beat Professional-Lev...,168,yjryrd,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,45,Paper: [https://arxiv.org/abs/2211.00241](http...,1.667353e+09
8,[N] On the detection of synthetic images gener...,2,ykoodd,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,0,Paper: [https://arxiv.org/abs/2211.00680](http...,1.667439e+09
9,[R] Instance segmentation of paper fiber networks,19,yk591e,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,I've been trying to perform instance segmentat...,1.667396e+09
